# Hyperasの練習

*20170715 Mitsuhiro Ito*


* (https://github.com/maxpumperla/hyperas/tree/master/examples)


---

In [1]:
from __future__ import print_function
import sys
from utils import save_model_viz, save_weights, save_hist, plot_hist
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import uniform, choice, conditional
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [3]:
def data():
    datagen = ImageDataGenerator(rescale=1.0 / 255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    train_generator = datagen.flow_from_directory('data/train/',
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary',
        shuffle=False)

    valid_generator = datagen.flow_from_directory('data/valid/',
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary',
        shuffle=False)
    return train_generator, valid_generator

In [4]:
def model(train_generator, valid_generator):

    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
              
    
          
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    batch_size = 64


    model.fit_generator(train_generator, 
                    steps_per_epoch = 2000 // batch_size, 
                    epochs=3, 
                    verbose=1, 
                    validation_data = valid_generator,
                    validation_steps= 800 // batch_size)
    
    loss,acc = model.evaluate_generator(train_generator, steps=64)
    
    print(loss, acc)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [5]:
train_generator, valid_generator = data()

best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=1,
                                      trials=Trials(),
                                      notebook_name='20170714_hyperas2' )
print("Evalutation of best performing model:")
print(best_model.evaluate_generator(train_generator, steps=64))
print(best_run)



FileNotFoundError: [Errno 2] No such file or directory: 'data/train/'

In [ ]:
if conditional({{choice(['three', 'four'])}}) == 'four':
        model.add(Conv2D([32, 64], (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))